In [1]:
from flask import Flask, render_template, request
from flask_cors import CORS
from werkzeug.utils import secure_filename
from geopy.geocoders import Nominatim
from datetime import datetime
import os
import pandas as pd
import json
from haversine import haversine
import re

app = Flask(__name__)
CORS(app)
geolocator = Nominatim(user_agent="my_geocoder")
UPLOAD_FOLDER1 = '/home/s60181908/tmp1'  # 업로드 파일을 저장할 디렉토리 설정 111
UPLOAD_FOLDER2 = '/home/s60181908/tmp2'  # 업로드 파일을 저장할 디렉토리 설정 222
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER1

# https://api-test.biblio19.net

def open_json():
    files = os.listdir(UPLOAD_FOLDER1)
    json_files = [file for file in files if file.endswith('.json')]
    
    if not json_files:
        return None

    latest_json_file = max(json_files, key=lambda x: os.path.getmtime(os.path.join(UPLOAD_FOLDER1, x)))
    file_path = os.path.join(UPLOAD_FOLDER1, latest_json_file)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
            return data
    except json.JSONDecodeError:
        return None

def open_json2():
    files = os.listdir(UPLOAD_FOLDER2)
    json_files = [file for file in files if file.endswith('.json')]
    
    if not json_files:
        return None

    latest_json_file = max(json_files, key=lambda x: os.path.getmtime(os.path.join(UPLOAD_FOLDER2, x)))
    file_path = os.path.join(UPLOAD_FOLDER2, latest_json_file)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
            return data
    except json.JSONDecodeError:
        return None

def is_json_file(filename):
    return filename.lower().endswith(".json")

def date_filtering(x):
    x_modified = x.replace('T', ' ')
    x_modified = x_modified.rsplit(':', 1)[0]
    return x_modified
def activity_and_stay(af):
    data_list = []
    timeline_objects = af['timelineObjects']

    for data in timeline_objects:
        if data.get('activitySegment', {}).get('activityType'):
            activity_segment = data.get('activitySegment', {})
            activity_type = activity_segment.get('activityType')
        
            if activity_type:
                start_location = activity_segment.get('startLocation', {})
                end_location = activity_segment.get('endLocation', {})
                duration = activity_segment.get('duration', {})
        
                start_latitude = start_location.get('latitudeE7', 0) / 1e7
                start_longitude = start_location.get('longitudeE7', 0) / 1e7
        
                end_latitude = end_location.get('latitudeE7', 0) / 1e7
                end_longitude = end_location.get('longitudeE7', 0) / 1e7
        
                start_timestamp = duration.get('startTimestamp', '')
                end_timestamp = duration.get('endTimestamp', '')
        
                filtering_start_time = date_filtering(start_timestamp)
                filtering_end_time = date_filtering(end_timestamp)
                
                segment_data = {
                    'Activity Type': activity_type,
                    '시작 시간': filtering_start_time,
                    '종료 시간': filtering_end_time,
                    '시작 위치(위도)': start_latitude,
                    '시작 위치(경도)': start_longitude,
                    '종료 위치(위도)': end_latitude,
                    '종료 위치(경도)': end_longitude
                }
            data_list.append(segment_data)
            
        elif data.get('placeVisit', {}).get('placeVisitType'):
            stay_segment = data.get('placeVisit', {})
            stay_type = stay_segment.get('placeVisitType')

            if stay_type:
                start_location = stay_segment.get('location', {})
                end_location = stay_segment.get('location', {})
                duration = stay_segment.get('duration', {})
        
                start_latitude = start_location.get('latitudeE7', 0) / 1e7
                start_longitude = start_location.get('longitudeE7', 0) / 1e7

                end_latitude = end_location.get('latitudeE7', 0) / 1e7
                end_longitude = end_location.get('longitudeE7', 0) / 1e7
        
                start_timestamp = duration.get('startTimestamp', '')
                end_timestamp = duration.get('endTimestamp', '')
        
                filtering_start_time = date_filtering(start_timestamp)
                filtering_end_time = date_filtering(end_timestamp)

                segment_data = {
                    'Activity Type': stay_type,
                    '시작 시간': filtering_start_time,
                    '종료 시간': filtering_end_time,
                    '시작 위치(위도)': start_latitude,
                    '시작 위치(경도)': start_longitude,
                    '종료 위치(위도)': end_latitude,
                    '종료 위치(경도)': end_longitude
                }
            data_list.append(segment_data)
            
    return data_list

def make_meeting_points(df1, df2) :
    matches = []
    
    for idx1, row1 in df1.iterrows():
        for idx2, row2 in df2.iterrows():
            if (row1['시작 시간'] <= row2['시작 시간'] and row1['종료 시간'] >= row2['시작 시간']) or (row1['시작 시간'] <= row2['종료 시간'] and row1['종료 시간'] >= row2['종료 시간']):
                start_distance = haversine((row1['시작 위치(위도)'], row1['시작 위치(경도)']), (row2['시작 위치(위도)'], row2['시작 위치(경도)']), unit='m')
                end_distance = haversine((row1['종료 위치(위도)'], row1['종료 위치(경도)']), (row2['종료 위치(위도)'], row2['종료 위치(경도)']), unit='m')
                if start_distance <= 20 or end_distance <= 20:
                    start_time = max(row1['시작 시간'], row2['시작 시간'])
                    end_time = min(row1['종료 시간'], row2['종료 시간'])
                    new_match = f"{start_time} ~ {end_time} 시간에 {row1['시작 위치(위도)']}, {row1['시작 위치(경도)']} 위치에서 만났을 수 있다"
                    if new_match not in matches:
                        matches.append(new_match)
    
    return matches

def data_to_math(data_list) :
    result_list = []
    
    for item in data_list:
        match = re.search(r'(\d+\.\d+), (\d+\.\d+)', item)
        if match:
            latitude, longitude = map(float, match.groups())
            result_list.append({'latitude': latitude, 'longitude': longitude})

    json_list = json.dumps(result_list)
    
    return json_list

In [2]:
a=open_json()
b=open_json2()

cf1 = activity_and_stay(a)
cf2 = activity_and_stay(b)

df1 = pd.DataFrame(cf1)
df2 = pd.DataFrame(cf2)

meeting_points = make_meeting_points(df1, df2)
meeting_points

['2023-10-04 07:21 ~ 2023-10-04 07:44 시간에 37.222691, 127.190167 위치에서 만났을 수 있다',
 '2023-10-05 03:35 ~ 2023-10-05 06:34 시간에 37.222691, 127.190167 위치에서 만났을 수 있다',
 '2023-10-11 02:40 ~ 2023-10-11 02:58 시간에 37.222691, 127.190167 위치에서 만났을 수 있다',
 '2023-10-11 03:23 ~ 2023-10-11 03:38 시간에 37.222691, 127.190167 위치에서 만났을 수 있다',
 '2023-10-12 03:31 ~ 2023-10-12 06:53 시간에 37.222691, 127.190167 위치에서 만났을 수 있다',
 '2023-10-18 03:18 ~ 2023-10-18 03:50 시간에 37.222691, 127.190167 위치에서 만났을 수 있다']

In [3]:
data = data_to_math(meeting_points)
data

'[{"latitude": 37.222691, "longitude": 127.190167}, {"latitude": 37.222691, "longitude": 127.190167}, {"latitude": 37.222691, "longitude": 127.190167}, {"latitude": 37.222691, "longitude": 127.190167}, {"latitude": 37.222691, "longitude": 127.190167}, {"latitude": 37.222691, "longitude": 127.190167}]'

In [1]:
from flask import Flask, render_template, request
from flask_cors import CORS
from werkzeug.utils import secure_filename
from geopy.geocoders import Nominatim
from datetime import datetime
import os
import pandas as pd
import json
from haversine import haversine
import re

app = Flask(__name__)
CORS(app)
geolocator = Nominatim(user_agent="my_geocoder")
UPLOAD_FOLDER1 = '/home/s60181908/tmp1'  # 업로드 파일을 저장할 디렉토리 설정 111
UPLOAD_FOLDER2 = '/home/s60181908/tmp2'  # 업로드 파일을 저장할 디렉토리 설정 222
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER1

# https://api-test.biblio19.net

def open_json():
    files = os.listdir(UPLOAD_FOLDER1)
    json_files = [file for file in files if file.endswith('.json')]
    
    if not json_files:
        return None

    latest_json_file = max(json_files, key=lambda x: os.path.getmtime(os.path.join(UPLOAD_FOLDER1, x)))
    file_path = os.path.join(UPLOAD_FOLDER1, latest_json_file)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
            return data
    except json.JSONDecodeError:
        return None

def open_json2():
    files = os.listdir(UPLOAD_FOLDER2)
    json_files = [file for file in files if file.endswith('.json')]
    
    if not json_files:
        return None

    latest_json_file = max(json_files, key=lambda x: os.path.getmtime(os.path.join(UPLOAD_FOLDER2, x)))
    file_path = os.path.join(UPLOAD_FOLDER2, latest_json_file)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
            return data
    except json.JSONDecodeError:
        return None

def is_json_file(filename):
    return filename.lower().endswith(".json")

def date_filtering(x):
    x_modified = x.replace('T', ' ')
    x_modified = x_modified.rsplit(':', 1)[0]
    return x_modified

def activity_and_stay(af):
    data_list = []
    timeline_objects = af['timelineObjects']

    for data in timeline_objects:
        if data.get('activitySegment', {}).get('activityType'):
            activity_segment = data.get('activitySegment', {})
            activity_type = activity_segment.get('activityType')
        
            if activity_type:
                start_location = activity_segment.get('startLocation', {})
                end_location = activity_segment.get('endLocation', {})
                duration = activity_segment.get('duration', {})
        
                start_latitude = start_location.get('latitudeE7', 0) / 1e7
                start_longitude = start_location.get('longitudeE7', 0) / 1e7
        
                end_latitude = end_location.get('latitudeE7', 0) / 1e7
                end_longitude = end_location.get('longitudeE7', 0) / 1e7
        
                start_timestamp = duration.get('startTimestamp', '')
                end_timestamp = duration.get('endTimestamp', '')
        
                filtering_start_time = date_filtering(start_timestamp)
                filtering_end_time = date_filtering(end_timestamp)
                
                segment_data = {
                    'Activity Type': activity_type,
                    '시작 시간': filtering_start_time,
                    '종료 시간': filtering_end_time,
                    '시작 위치(위도)': start_latitude,
                    '시작 위치(경도)': start_longitude,
                    '종료 위치(위도)': end_latitude,
                    '종료 위치(경도)': end_longitude
                }
            data_list.append(segment_data)
            
        elif data.get('placeVisit', {}).get('placeVisitType'):
            stay_segment = data.get('placeVisit', {})
            stay_type = stay_segment.get('placeVisitType')

            if stay_type:
                start_location = stay_segment.get('location', {})
                end_location = stay_segment.get('location', {})
                duration = stay_segment.get('duration', {})
        
                start_latitude = start_location.get('latitudeE7', 0) / 1e7
                start_longitude = start_location.get('longitudeE7', 0) / 1e7

                end_latitude = end_location.get('latitudeE7', 0) / 1e7
                end_longitude = end_location.get('longitudeE7', 0) / 1e7
        
                start_timestamp = duration.get('startTimestamp', '')
                end_timestamp = duration.get('endTimestamp', '')
        
                filtering_start_time = date_filtering(start_timestamp)
                filtering_end_time = date_filtering(end_timestamp)

                segment_data = {
                    'Activity Type': stay_type,
                    '시작 시간': filtering_start_time,
                    '종료 시간': filtering_end_time,
                    '시작 위치(위도)': start_latitude,
                    '시작 위치(경도)': start_longitude,
                    '종료 위치(위도)': end_latitude,
                    '종료 위치(경도)': end_longitude
                }
            data_list.append(segment_data)
            
    return data_list

def make_meeting_points(df1, df2) :
    matches = []
    
    for idx1, row1 in df1.iterrows():
        for idx2, row2 in df2.iterrows():
            if (row1['시작 시간'] <= row2['시작 시간'] and row1['종료 시간'] >= row2['시작 시간']) or (row1['시작 시간'] <= row2['종료 시간'] and row1['종료 시간'] >= row2['종료 시간']):
                start_distance = haversine((row1['시작 위치(위도)'], row1['시작 위치(경도)']), (row2['시작 위치(위도)'], row2['시작 위치(경도)']), unit='m')
                end_distance = haversine((row1['종료 위치(위도)'], row1['종료 위치(경도)']), (row2['종료 위치(위도)'], row2['종료 위치(경도)']), unit='m')
                if start_distance <= 20 or end_distance <= 20:
                    start_time = max(row1['시작 시간'], row2['시작 시간'])
                    end_time = min(row1['종료 시간'], row2['종료 시간'])
                    new_match = f"{start_time} ~ {end_time} 시간에 {row1['시작 위치(위도)']}, {row1['시작 위치(경도)']} 위치에서 만났을 수 있다"
                    if new_match not in matches:
                        matches.append(new_match)
    
    return matches

def data_to_math(data_list) :
    result_list = []
    
    for item in data_list:
        match = re.search(r'(\d+\.\d+), (\d+\.\d+)', item)
        if match:
            latitude, longitude = map(float, match.groups())
            result_list.append({'latitude': latitude, 'longitude': longitude})

    json_list = json.dumps(result_list)
    
    return json_list
    
@app.route('/', methods=['GET'])
def index():
    return 'Hello World!'

@app.route('/upload_path1', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        files = request.files.getlist('file')  # 'file'라는 필드에서 파일 리스트 가져옴
        for file in files:
            if file:
                filename = secure_filename(file.filename)
                file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

        return "Files uploaded successfully"

    return '''
    <!doctype html>
    <title>File Upload</title>
    <h1>Upload Files</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file multiple>
      <input type=submit value=Upload>
    </form>
    '''


@app.route('/upload_path2', methods=['GET', 'POST'])
def upload_file2():
    if request.method == 'POST':
        files = request.files.getlist('file')  # 'file'라는 필드에서 파일 리스트 가져옴
        for file in files:
            if file:
                filename = secure_filename(file.filename)
                file.save(os.path.join(UPLOAD_FOLDER2, filename))

        return "Files uploaded successfully"

    return '''
    <!doctype html>
    <title>File Upload</title>
    <h1>Upload Files</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file multiple>
      <input type=submit value=Upload>
    </form>
    '''

    
@app.route('/upload_date', methods=['POST'])
def upload_date():
    if request.method == 'POST':
        date_str = request.form.get('date')
        try:
            date = datetime.strptime(date_str, "%Y-%m-%d")
            return f"날짜: {date}"
        except Exception as e:
            return f"에러 발생: {str(e)}"
    return render_template('index.html')

@app.route('/upload_address', methods=['POST'])
def upload_address():
    if request.method == 'POST':
        address = request.form.get('address')
        try:
            location = geolocator.geocode(address)
            if location:
                latitude = location.latitude
                longitude = location.longitude
            else:
                latitude = longitude = None
        
            return f"주소: {address}, 위도: {latitude}, 경도: {longitude}"
        except Exception as e:
            return f"에러 발생: {str(e)}"
    return render_template('index.html')


@app.route('/analyze', methods=['GET'])
def analyze():
    a = open_json()
    b = open_json2()

    cf1 = activity_and_stay(a)
    cf2 = activity_and_stay(b)
    
    df1 = pd.DataFrame(cf1)
    df2 = pd.DataFrame(cf2)
    
    meeting_points = make_meeting_points(df1, df2)
    
    data = data_to_math(meeting_points)
    return data

if __name__ == '__main__':
    app.run(host='0.0.0.0')


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'